In [9]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import numpy as np
import matplotlib.pyplot as plt
from dask import dataframe as dd
import csv
import gzip
import time

In [10]:
veri = pd.DataFrame(data=np.random.randint(99999, 99999999, size=(10000000,14)),columns=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14'])
veri['C15'] = pd.util.testing.rands_array(5,10000000)
veri.to_csv("new_data.csv")

D:\ProgramDosyalaari\anaconda\lib\site-packages\pandas\util\__init__.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


In [11]:
start = time.time()
veri = pd.read_csv('new_data.csv')
end = time.time()
print("loads the whole CSV file at once in the memory in a single dataframe: ",(end-start),"sec")

loads the whole CSV file at once in the memory in a single dataframe:  18.967334747314453 sec


In [12]:
start = time.time()
#read data in chunks of 1 million rows at a time
veri = pd.read_csv('new_data.csv',chunksize=1000000)
end = time.time()
print("Instead of reading the whole CSV at once, chunks of CSV are read into memory: ",(end-start),"sec")
pd_df = pd.concat(veri)

Instead of reading the whole CSV at once, chunks of CSV are read into memory:  0.2882671356201172 sec


In [13]:
start = time.time()
veri = pd.read_csv('new_data.csv',delimiter=',')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  17.80044388771057 sec


In [14]:
veri.head()

,Unnamed: 0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
0,0,75177999,17916186,4650004,19000906,42160012,18440951,53805842,94719520,74008856,52909114,43817160,80588537,73942922,22395207,Lx7kf
1,1,97878932,24690227,85375732,63175051,95031555,95460294,29592631,98861411,93237003,86688257,370361,52042690,76144433,46045778,04aSn
2,2,26538740,11404101,27220354,73108061,46194446,54232131,89367649,67237795,19599157,57918083,58311150,66638976,55947083,97968875,PG6aB
3,3,48185218,63589338,35275611,75073891,58758333,67237333,64410128,18723828,86078699,4955798,92281466,59838334,76475970,73647927,8GKHK
4,4,38084338,41904722,37930039,41618816,94557596,21679930,1743161,20049110,87990439,82871484,76776308,73388349,79112949,30952741,M4DeW


In [15]:
%%writefile testutility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [16]:
%%writefile testutility.py
import yaml
import logging
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df, table_cfg):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_cfg["columns"]))
    expected_col.sort()
    #df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if list(expected_col) == list(df.columns):
        print("column name validation passed")
        return 1
    else:
        print("column name validation failed")
        mismatch = list(set(df.columns).difference(expected_col))
        print("Columns not in YAML file: ", mismatch)
        missing =  list(set(expected_col).difference(df.columns))
        print("Columns not in data file: ", missing)
        return 0

Overwriting testutility.py


In [17]:
%%writefile numericdata.yaml
file_type: csv
dataset_name: file
file_name: new_data
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
      - C1
      - C2
      - C3
      - C4
      - C5
      - C6
      - C7
      - C8
      - C9
      - C10
      - C11
      - C12
      - C13
      - C14

Overwriting numericdata.yaml


In [18]:
# Read config file
import testutility as util
config_data = util.read_config_file("numericdata.yaml")

In [19]:
config_data['inbound_delimiter']

','

In [20]:
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'new_data',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['C1',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'C14']}

In [21]:
df_sample = pd.read_csv("new_data.csv",delimiter=',')
df_sample.head(3)

,Unnamed: 0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
0,0,75177999,17916186,4650004,19000906,42160012,18440951,53805842,94719520,74008856,52909114,43817160,80588537,73942922,22395207,Lx7kf
1,1,97878932,24690227,85375732,63175051,95031555,95460294,29592631,98861411,93237003,86688257,370361,52042690,76144433,46045778,04aSn
2,2,26538740,11404101,27220354,73108061,46194446,54232131,89367649,67237795,19599157,57918083,58311150,66638976,55947083,97968875,PG6aB


In [22]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

D:\ProgramDosyalaari\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
0,0,75177999,17916186,4650004,19000906,42160012,18440951,53805842,94719520,74008856,52909114,43817160,80588537,73942922,22395207,Lx7kf
1,1,97878932,24690227,85375732,63175051,95031555,95460294,29592631,98861411,93237003,86688257,370361,52042690,76144433,46045778,04aSn
2,2,26538740,11404101,27220354,73108061,46194446,54232131,89367649,67237795,19599157,57918083,58311150,66638976,55947083,97968875,PG6aB
3,3,48185218,63589338,35275611,75073891,58758333,67237333,64410128,18723828,86078699,4955798,92281466,59838334,76475970,73647927,8GKHK
4,4,38084338,41904722,37930039,41618816,94557596,21679930,1743161,20049110,87990439,82871484,76776308,73388349,79112949,30952741,M4DeW


In [27]:
#Output gz file
df.to_csv('new_datas.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

In [26]:
import os
#Size of the file
os.path.getsize('new_data.csv')

1393753171

In [28]:
#Size of the file
os.path.getsize('new_datas.csv.gz')

738416706